In [ ]:
import requests
from dotenv import load_dotenv

## 설정값 정리 ###
load_dotenv()


azure_oai_endpoint = oai_endpoint
azure_oai_key = 

azure_search_endpoint = 
azure_search_key = 
azure_search_indexname = 
azure_search_semanticname = 

def request_gpt(prompt):
    #endpoint
    endpoint = azure_oai_endpoint
    #headers
    headers = {
        "Content-Type":"application/json",
        "api-key":azure_oai_key
    }
    #body
    body = {
        "messages": [           
            {
                "role": "system",
                "content": "너는 여행전문가야."
            },
            {
                "role": "assistant",
                "content": "네 ~"
            },            
            {
                "role": "user",
                "content":  prompt
            },
        ],
        "temperature": 0.7,
        "top_p": 0.95,
        "max_tokens": 1000,
        "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": azure_search_endpoint,
                "semantic_configuration": azure_search_semanticname,
                "query_type": "semantic",
                "fields_mapping": {},
                "in_scope": True,
                "filter": None,
                "strictness": 3,
                "top_n_documents": 5,
                "key": azure_search_key,
                "indexName": azure_search_indexname
            }
        }
    ]
    }

    response = requests.post(endpoint, headers=headers, json=body)

    if response.status_code == 200:
        response_json = response.json()
        content_text = response_json['choices'][0]['message']['content']
        citations = response_json['choices'][0]['message']['context']['citations']
        return content_text, citations
    else:
        return response.status_code, response.text

request_result1, request_result2  = request_gpt("London 가르쳐줘")

print(request_result1)
print(request_result2)


In [ ]:
## 설정값 정리 ###


azure_oai_endpoint = ""
azure_oai_key = ""

azure_search_endpoint = ""
azure_search_key = ""
azure_search_indexname = ""
azure_search_semanticname = ""

In [ ]:
import os
import json
from dotenv import load_dotenv

# Add OpenAI import
from openai import AzureOpenAI

def main(): 
    try:
        # Flag to show citations
        show_citations = True

        # Get configuration settings 
        load_dotenv()
        azure_oai_endpoint = os.getenv("AZURE_OAI_ENDPOINT")
        azure_oai_key = os.getenv("AZURE_OAI_KEY")
        azure_oai_deployment = os.getenv("AZURE_OAI_DEPLOYMENT")
        azure_search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
        azure_search_key = os.getenv("AZURE_SEARCH_KEY")
        azure_search_index = os.getenv("AZURE_SEARCH_INDEX")
        
        # Initialize the Azure OpenAI client
        client = AzureOpenAI(
            base_url=azure_oai_endpoint,
            api_key=azure_oai_key,
            api_version="2024-02-15-preview"
        )

        # Get the prompt
        text = input('\nEnter a question:\n')

        # Configure your data source with corrected keys and type
        extension_config = {
            "data_sources": [
                {
                    "type": "azure_search",  
                    "parameters": {
                        "endpoint": azure_search_endpoint,
                        "index_name": azure_search_index,
                        "authentication": {
                            "type": "api_key",
                            "api_key": azure_search_key
                        }
                    }
                }
            ]
        }

        # Send request to Azure OpenAI model
        print("...Sending the following request to Azure OpenAI endpoint...")
        print("Request: " + text + "\n")

        response = client.chat.completions.create(
            model=azure_oai_deployment,
            temperature=0.5,
            max_tokens=1000,
            messages=[
                {"role": "system", "content": "You are a helpful travel agent"},
                {"role": "user", "content": text}
            ],
            extra_body=extension_config
        )

        # Print response
        print("Response: " + response.choices[0].message.content + "\n")

        if show_citations:
            # Print citations
            print("Citations:")
            citations = response.choices[0].message.context["messages"][0]["content"]
            citation_json = json.loads(citations)
            for c in citation_json["citations"]:
                print("  Title: " + c['title'] + "\n    URL: " + c['url'])

    except Exception as ex:
        print(ex)

if __name__ == '__main__': 
    main()


In [ ]:
print(type(azure_search_key))

In [ ]:
n = 0
for citation in request_result2:
    n+=1
    print(f'doc {n}:\n',citation['content'])
    print('*'*30)

In [ ]:
print(request_result['choices'][0]['message']['content'])

In [ ]:
request_result.status_code, request_result.json()